<a href="https://colab.research.google.com/github/MaidinuerSaimi/Python-courses/blob/main/%E2%80%9CWorkshop_2_2_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 2.2  (Wrap up quiz M2)

Open the dataset `blood_transfusion.csv` with the following command:

In [ ]:
import pandas as pd

blood_transfusion = pd.read_csv("https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/datasets/blood_transfusion.csv")
target_name = "Class"
data = blood_transfusion.drop(columns=target_name)
target = blood_transfusion[target_name]

`blood_transfusion` is a `pandas` dataframe. The column `“Class”` contains the target variable.

<div class="alert alert-info">

## Question 1

Select the correct answers from the following proposals.
***
1. The problem to be solved is a regression problem
2. The problem to be solved is a binary classification problem (exactly 2 possible classes)
3. The problem to be solved is a multiclass classification problem (more than 2 possible classes)
4. The proportions of the class counts are imbalanced: some classes have more than twice as many rows than others

Select all answers that apply

Hint: `target.unique()`, and `target.value_counts()` are methods that are helpful to answer to this question.

<div class="alert alert-info">

## Question 2
    
Using a `sklearn.dummy.DummyClassifier` and the strategy `"most_frequent"`, what is the average of the accuracy scores obtained by performing a 10-fold cross-validation?  ~25%, ~50% or ~75%?

Hint: You can check the documentation of `sklearn.model_selection.cross_val_score` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) and `sklearn.model_selection.cross_validate` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html).

<div class="alert alert-info">

## Question 3
    
Repeat the previous experiment but compute the balanced accuracy instead of the accuracy score. Pass `scoring="balanced_accuracy"` when calling `cross_validate` or `cross_val_score` functions.  What is the mean score now?    

We will use a `sklearn.neighbors.KNeighborsClassifier` for the remainder of this workshop.

<div class="alert alert-info">

## Question 4

Why is it relevant to add a preprocessing step to scale the data using a StandardScaler when working with a KNeighborsClassifier?
***
1. faster to compute the list of neighbors on scaled data

2. k-nearest neighbors is based on computing some distances. Features need to be normalized to contribute approximately equally to the distance computation.

3. This is irrelevant. One could use k-nearest neighbors without normalizing the dataset and get a very similar cross-validation score.




Create a scikit-learn pipeline (using `sklearn.pipeline.make_pipeline`) where a `StandardScaler` will be used to scale the data followed by a `KNeighborsClassifier`. Use the default hyperparameters.

<div class="alert alert-info">

## Question 5

Inspect the parameters of the created pipeline. What is the value of `K`, the number of neighbors considered when predicting with the k-nearest neighbors.  Is it 1, 3, 5, 8 or 10?

Hint: You can use `model.get_params()` to get the parameters of a scikit-learn estimator.

<div class="alert alert-info">

## Question 6

Set `n_neighbors=1` in the previous model and evaluate it using a 10-fold cross-validation. Use the balanced accuracy as a score. What can you say about this model? Compare the average of the train and test scores to argument your answer.

**
1. The model clearly underfits

2. The model generalizes

3. The model clearly overfits

Hint: compute the average test score and the average train score and compare them. Make sure to pass `return_train_score=True` to the cross_validate function to also compute the train score.

We will now study the effect of the parameter `n_neighbors` on the train and test score using a validation curve. You can use the following parameter range:

```python
param_range = [1, 2, 5, 10, 20, 50, 100, 200, 500]
```

Also, use a 5-fold cross-validation and compute the balanced accuracy score instead of the default accuracy score (check the scoring parameter). Finally, plot the average train and test scores for the different value of the hyperparameter. We recall that the name of the parameter can be found using `model.get_params()`.

<div class="alert alert-info">

## Question 7

For each of the parameter ranges 1-10, 10-100 and 100-500, answer the following:

***
1. Does the model underfit in this parameter range?
2. Does the model overfit in this parameter range?
3. Does the model best generalize in this parameter range?


# Wrap up

Overfitting is caused by the limited size of the training set, the noise in the data, and the high flexibility of common machine learning models.

Underfitting happens when the learnt prediction functions suffer from systematic errors. This can be caused by a choice of model family and parameters, which leads to a lack of flexibility to capture the repeatable structure of the true data generating process.

For a fixed training set, the objective is to minimize the test error by adjusting the model family and its parameters to find the best trade-off between overfitting for underfitting.

For a given choice of model family and parameters, increasing the training set size will decrease overfitting but can also cause an increase of underfitting.

The test error of a model that is neither overfitting nor underfitting can still be high if the variations of the target variable cannot be fully determined by the input features. This irreducible error is caused by what we sometimes call label noise. In practice, this often happens when we do not have access to important features for one reason or another.
